<a href="https://colab.research.google.com/github/absolutelydawn/TestNeoX/blob/main/Learn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall transformers -y
!pip install transformers
!pip install accelerate
!pip install accelerate --upgrade

# Colab에 "git"을 설치하고 레포지토리를 클론
!pip install gitpython
import git

repo_url = "https://github.com/songys/Chatbot_data.git"
destination_folder = "/content/Chatbot_data"

git.Git("/content").clone(repo_url)

# 다운로드 된 데이터 확인
import os

data_folder = "/content/Chatbot_data"
data_files = os.listdir(data_folder)
print(data_files)

!pip install accelerate -U

import os

data_folder = "/content/Chatbot_data"
data_files = os.listdir(data_folder)
print(data_files)

import pandas as pd
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, GPT2Config, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 데이터 파일 경로 설정
data_file_path = "/content/Chatbot_data/ChatbotData.csv"

# CSV 파일을 pandas DataFrame으로 불러오기
df = pd.read_csv(data_file_path)

# 데이터 확인 (예시)
print(df.head())

# 입력 데이터와 출력 데이터 분리
input_texts = df['Q'].tolist()
output_texts = df['A'].tolist()

print("Input texts sample:")
print(input_texts[:5])

print("\nOutput texts sample:")
print(output_texts[:5])

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', pad_token='<pad>')

import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, AutoTokenizer, GPT2Config, TrainingArguments, Trainer

# 모델 설정
config = GPT2Config.from_pretrained('skt/kogpt2-base-v2', pad_token_id=3)

# 토큰화
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', pad_token='<pad>')

# 모델 생성
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', config=config)

# 입력 텍스트와 출력 텍스트 준비
input_texts = ['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
output_texts = ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']

# 입력 텍스트와 출력 텍스트를 토큰화
input_encodings = tokenizer(input_texts + output_texts, truncation=True, padding=True)

# 입력 텍스트와 출력 텍스트의 인코딩 분할
input_encodings_split = {'input_ids': input_encodings['input_ids'][:len(input_texts)], 'attention_mask': input_encodings['attention_mask'][:len(input_texts)]}
output_encodings_split = {'input_ids': input_encodings['input_ids'][len(input_texts):], 'attention_mask': input_encodings['attention_mask'][len(input_texts):]}


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_encodings, output_encodings):
        self.input_encodings = input_encodings
        self.output_encodings = output_encodings

    def __getitem__(self, idx):
        return {'input_ids': self.input_encodings['input_ids'][idx],
                'attention_mask': self.input_encodings['attention_mask'][idx],
                'labels': self.output_encodings['input_ids'][idx]
               }

    def __len__(self):
        return len(self.input_encodings['input_ids'])

train_dataset = CustomDataset(input_encodings_split, output_encodings_split)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 경로 설정
    num_train_epochs=5,              # 전체 학습 에포크 수
    per_device_train_batch_size=2,   # 디바이스당 배치 크기
    save_total_limit=3,              # 저장되는 체크포인트 숫자 제한
    warmup_steps=500,                # 학습률 스케줄 관련: 웜업 스텝 설정
    weight_decay=0.01,               # 학습률 스케줄 관련: 가중치 감쇠 적용
)

# 트레이너 생성 및 학습 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# 학습 진행
trainer.train()

# 학습된 모델 저장
model.save_pretrained('./trained_model')

#학습 완료 후 모델 확인하기

# Step1 : 학습 모델 불러오기
model = GPT2LMHeadModel.from_pretrained('./trained_model')
model.eval()  # 모델을 평가(추론) 모드로 전환

# Step2 : 입력 텍스트 토큰화
input_text = "요즘 날씨가 너무 더워서 밖에 안나가고 싶어"
input_tokens = tokenizer.encode(input_text, return_tensors="pt")

# Step3 : 답변 생성하기
with torch.no_grad():  # 기울기 계산(그래디언트)을 비활성화하여 메모리 사용량을 줄입니다.
    output_tokens = model.generate(input_tokens, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(decoded_output)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
['data.png', 'gemimage.png', 'ChatbotData.csv', 'LICENSE', 'README.md', '.git']
['data.png', 'gemimage.png', 'ChatbotData.csv', 'LICENSE', 'README.md', '.git']
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
Input texts sample:
['12시 땡!', '1지망 학교 떨어졌어', 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


요즘 날씨가 너무 더워서 밖에 안나가고 싶어서
이렇게 #아기옷 으로 겟겟했어요
#육아스타그램 baby_boy.
#베이비그램
#아들스타 


In [3]:
# 입력 데이터 받아서 대화 형식으로 출력하기

def chat_with_model(input_text):
    # Step1: 입력 텍스트 토큰화
    input_tokens = tokenizer.encode(input_text, return_tensors="pt")

    # Step2: 답변 생성하기
    with torch.no_grad():
        output_tokens = model.generate(
            input_tokens, max_length=500, num_return_sequences=1, no_repeat_ngram_size=2
        )

    decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return decoded_output


while True:
    user_input = input("User: ")
    if user_input.lower() == "quit":
        break

    response = chat_with_model(user_input)
    print("Model: ", response)

User: 안녕하세요?
Model:  안녕하세요?"
"그럼, 그건 뭐예요!"
그녀는 고개를 끄덕였다.
"아니, 그게 무슨 소리예요."
그는 고개를 저었다.
"이제 그만 가봐야겠어요. 그걸로 끝이에요. 그리고 그 다음에 다시 한 번 더 말씀드리죠."
"그래, 그럼, 이제 그만."
그녀가 다시 말했다.
"그러면 그 다음엔 다시 한번 더 말해주세요. 그럼 그 다음에는 다시 다시 말씀해주시죠. 그리고 다시 말해서 그 다음은 다시 말해주세요, 그 뒤에 다시."
그는 다시 고개를 들었다.
"네, 알겠습니다."
그러자 그녀는 다시 말을 이었다.
"그리고 다시 또 말씀드릴게요. 이번에 다시 얘기해주실 수 있는 건, 이게 뭐죠? 그 전에 다시 이야기해주시는 건가요요"
이번에는 그녀가 다시 물었다.
"아, 아니에요. 하지만 그 전에는 다시 이야기를 해주시지 않았어요" 하고 그녀는 말했다.
그녀의 대답은 단호했다.
"하지만 그 후에 다시 설명해주셔야죠." 그녀는 대답했다.
"다시 한번 말씀해 주세요." 그녀는 고개를 흔들었다.
그가 다시 대답했다.
그때 그녀는 그의 말을 가로막았다.
"왜, 왜, 어떻게 해야 하는지 모르겠어요, 이거예요, 아까 말씀하신 대로."
그러고는 다시 입을 열었다.
"당신은 이걸 어떻게 할 거예요?
"알겠습니다, 알았어. 하지만 당신은 그 일을 어떻게 해요"라고 그녀는 말했다. 만약 당신이 그 일에 대해 알고 있다면 그 일은 어떻게 될까?
그렇다면 당신에게는 어떤 일이 일어날까?
"어떻게 해야 할까요"라는 질문에 그녀는 대답하지 않았다.
"만약 당신을 도와줄 수 있다면 어떻게 하겠어?" 그는 말했다.
그러고 나서 그녀는 말을 계속했다.
"나는 당신에게 도움을 줄 수 있을 거라고 생각했어."
그리고 그녀는 이렇게 말했다.
그리고는 다시 덧붙였다.
그것은 당신과 함께 있는 것이 아니라 당신으로 하여금 당신에 대해 이야기하도록 하는 것이었다.
그런데 그 일이 일어난 것은 바로 그 다음날이었다.
그날 밤 그녀는 당신의 집에 있는 것을 알고 있었다.
그곳에는 당신도

In [ ]:
# 추가 학습

def generate_from_trial_tokenized(tokenized):
    tokenized = tokenized.unsqueeze(0)  # 배치 차원 추가
    tokenized = tokenized.to(device)  # 텐서를 GPU로 이동
    model.to(device)  # 모델을 GPU로 이동

!pip uninstall transformers -y
!pip install transformers
!pip install accelerate
!pip install accelerate --upgrade

import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

import urllib.request
url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
urllib.request.urlretrieve(url, "ChatbotData.csv")

# 데이터 불러오기
data_file_path = "ChatbotData.csv"
df = pd.read_csv(data_file_path)

# 입력과 출력을 라벨없이 합친 텍스트로 설정
all_texts = df['Q'].tolist() + df['A'].tolist()

# 토큰화
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
all_encodings = tokenizer(all_texts, truncation=True, padding=True)

class UnsupervisedDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

unsupervised_dataset = UnsupervisedDataset(all_encodings)

# 기존 학습된 모델 경로를 설정합니다.
pretrained_model_path = "gpt2"

# 기존 학습된 모델 불러오기
config = GPT2Config.from_pretrained(pretrained_model_path)
config.gradient_checkpointing = True
model = GPT2LMHeadModel.from_pretrained(pretrained_model_path, config=config)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./finetuned_model',
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 트레이너 생성 및 학습 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=unsupervised_dataset
)

# 추가 학습 진행
trainer.train()

# 추가 학습된 모델 저장
model.save_pretrained('./finetuned_model')


Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,2.178300
200,1.980600
300,1.887200
400,1.819100
500,1.750800
600,1.705800
700,1.694200
800,1.666300
900,1.606400
1000,1.627900


In [5]:
def generate_from_trial_tokenized(tokenized):
    tokenized = tokenized.unsqueeze(0)  # 배치 차원 추가
    tokenized = tokenized.to(device)  # 텐서를 GPU로 이동
    model.to(device)  # 모델을 GPU로 이동

# 챗봇 테스트
def chat_with_model(input_text):
    input_tokens = tokenizer.encode(input_text, return_tensors="pt")

    with torch.no_grad():
        output_tokens = model.generate(
            input_tokens, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2
        )

    decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return decoded_output

while True:
    user_input = input("User: ")
    if user_input.lower() == "quit":
        break

    response = chat_with_model(user_input)
    print("Model: ", response)


User: 안녕하세요?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: ignored